In [59]:
import numpy as np

In [60]:
def get_node_id(i_row, i_col, i_layer, n_row, n_col, n_layer = None):
    """
    根据行列层编号获取节点编号
    :param i_row: 行编号
    :param i_col: 列编号
    :param i_layer: 层编号
    :param n_row: 行数
    :param n_col: 列数
    :param n_layer: 层数
    :return: 节点编号
    """
    return i_layer*n_row*n_col + i_row*n_col + i_col + 1
def create_msh_from_array(x_array, y_array, z_array, file_path):
    """
    从xyz的坐标数组中创建结构化msh网格
    :param x_array:x的一维数组，shape=(n_col,)
    :param y_array:y的一维数组，shape=(n_row,)
    :param z_array:z的三维数组，shape=(n_row,n_col,n_layer)
    :param file_path:文件路径
    """
    n_row = z_array.shape[0]
    n_col = z_array.shape[1]
    n_layer = z_array.shape[2]
    # 检查输入的数组是否符合要求
    if x_array.shape[0] != n_col:
        raise ValueError(f'x_array.shape[0] != n_col({n_col})')
    if y_array.shape[0] != n_row:
        raise ValueError(f'y_array.shape[0] != n_row({n_row})')
    
    # 按照顺序给每个节点编号，存入临时字符串变量
    nodes_str = ''
    for i_layer in range(n_layer):
        for i_row in range(n_row):
            for i_col in range(n_col):
                node_id = get_node_id(i_row, i_col, i_layer, n_row, n_col, n_layer)
                nodes_str += f'{node_id} {x_array[i_col]} {y_array[i_row]} {z_array[i_row,i_col,i_layer]}\n'
    # print('nodes_str:\n', nodes_str)
    # 按照顺序给每个体单元编号，存入临时字符串变量
    elements_str = ''
    element_id = 1
    for i_layer in range(n_layer-1):
        for i_row in range(n_row-1):
            for i_col in range(n_col-1):
                # 体单元的8个节点编号
                node_id_1 = get_node_id(i_row, i_col, i_layer, n_row, n_col, n_layer)
                node_id_2 = node_id_1 + 1
                node_id_3 = node_id_2 + n_col
                node_id_4 = node_id_1 + n_col
                node_id_5 = node_id_1 + n_row*n_col
                node_id_6 = node_id_2 + n_row*n_col
                node_id_7 = node_id_3 + n_row*n_col
                node_id_8 = node_id_4 + n_row*n_col
                elements_str += f'{element_id} 5 2 0 1 {node_id_1} {node_id_2} {node_id_3} {node_id_4} {node_id_5} {node_id_6} {node_id_7} {node_id_8}\n'
                element_id += 1
    # print('elements_str:\n', elements_str)
    # 按照顺序给每个底面单元编号，存入临时字符串变量
    bottom_elements_str = ''
    for i_row in range(n_row-1):
        for i_col in range(n_col-1):
            # 底面单元的4个节点编号
            node_id_1 = get_node_id(i_row, i_col, 0, n_row, n_col, n_layer)
            node_id_2 = node_id_1 + 1
            node_id_3 = node_id_2 + n_col
            node_id_4 = node_id_1 + n_col
            bottom_elements_str += f'{element_id} 3 2 1 1 {node_id_1} {node_id_2} {node_id_3} {node_id_4}\n' # 第一个1表示底面边界条件组，第二个1表示地面几何组
            element_id += 1
    # print('bottom_elements_str:\n', bottom_elements_str)
    # 按照顺序给每个zmax单元（即i_layer为n_layer-1的面）编号，存入临时字符串变量
    zmax_elements_str = ''
    for i_row in range(n_row-1):
        for i_col in range(n_col-1):
            # zmax单元的4个节点编号
            node_id_1 = get_node_id(i_row, i_col, n_layer-1, n_row, n_col, n_layer)
            node_id_2 = node_id_1 + 1
            node_id_3 = node_id_2 + n_col
            node_id_4 = node_id_1 + n_col
            zmax_elements_str += f'{element_id} 3 2 2 2 {node_id_1} {node_id_2} {node_id_3} {node_id_4}\n'
            element_id += 1
    # 按照顺序给每个xmin单元（即n_col为0的面）编号，存入临时字符串变量
    xmin_elements_str = ''
    for i_layer in range(n_layer-1):
        for i_row in range(n_row-1):
            # xmin单元的4个节点编号
            node_id_1 = get_node_id(i_row, 0, i_layer, n_row, n_col, n_layer)
            node_id_2 = node_id_1 + n_row*n_col
            node_id_3 = node_id_2 + n_col
            node_id_4 = node_id_1 + n_col
            xmin_elements_str += f'{element_id} 3 2 3 3 {node_id_1} {node_id_2} {node_id_3} {node_id_4}\n' # 第一个1表示底面边界条件组，第二个1表示地面几何组
            element_id += 1
    # print('xmin_elements_str:\n', xmin_elements_str)
    # 按照顺序给每个xmax单元（即n_col为n_col-1的面）编号，存入临时字符串变量
    xmax_elements_str = ''
    for i_layer in range(n_layer-1):
        for i_row in range(n_row-1):
            # xmax单元的4个节点编号
            node_id_1 = get_node_id(i_row, n_col-1, i_layer, n_row, n_col, n_layer)
            node_id_2 = node_id_1 + n_row*n_col
            node_id_3 = node_id_2 + n_col
            node_id_4 = node_id_1 + n_col
            xmax_elements_str += f'{element_id} 3 2 4 4 {node_id_1} {node_id_2} {node_id_3} {node_id_4}\n' # 第一个1表示底面边界条件组，第二个1表示地面几何组
            element_id += 1
    # print('xmax_elements_str:\n', xmax_elements_str)
    # 按照顺序给每个ymin单元（即n_row为0的面）编号，存入临时字符串变量
    ymin_elements_str = ''
    for i_layer in range(n_layer-1):
        for i_col in range(n_col-1):
            # ymin单元的4个节点编号
            node_id_1 = get_node_id(0, i_col, i_layer, n_row, n_col, n_layer)
            node_id_2 = node_id_1 + n_row*n_col
            node_id_3 = node_id_2 + 1
            node_id_4 = node_id_1 + 1
            ymin_elements_str += f'{element_id} 3 2 5 5 {node_id_1} {node_id_2} {node_id_3} {node_id_4}\n' # 第一个1表示底面边界条件组，第二个1表示地面几何组
            element_id += 1
    # print('ymin_elements_str:\n', ymin_elements_str)
    # 按照顺序给每个ymax单元（即n_row为n_row-1的面）编号，存入临时字符串变量
    ymax_elements_str = ''
    for i_layer in range(n_layer-1):
        for i_col in range(n_col-1):
            # ymax单元的4个节点编号
            node_id_1 = get_node_id(n_row-1, i_col, i_layer, n_row, n_col, n_layer)
            node_id_2 = node_id_1 + n_row*n_col
            node_id_3 = node_id_2 + 1
            node_id_4 = node_id_1 + 1
            ymax_elements_str += f'{element_id} 3 2 6 6 {node_id_1} {node_id_2} {node_id_3} {node_id_4}\n' # 第一个1表示底面边界条件组，第二个1表示地面几何组
            element_id += 1
    # print('ymax_elements_str:\n', ymax_elements_str)
    
    # 写入filepath
    with open(file_path, 'w') as f:
        f.write('$MeshFormat\n')
        f.write('2.2 0 8\n')
        f.write('$EndMeshFormat\n')
        f.write('$PhysicalNames\n')
        f.write('6\n')
        f.write('2 1 "terrain"\n')
        f.write('2 2 "zmax"\n')
        f.write('2 3 "xmin"\n')
        f.write('2 4 "xmax"\n')
        f.write('2 5 "ymin"\n')
        f.write('2 6 "ymax"\n')
        f.write('$EndPhysicalNames\n')
        f.write('$Nodes\n')
        f.write(f'{n_row*n_col*n_layer}\n')
        f.write(nodes_str)
        f.write('$EndNodes\n')
        f.write('$Elements\n')
        f.write(f'{element_id-1}\n')
        f.write(elements_str)
        f.write(bottom_elements_str)
        f.write(zmax_elements_str)
        f.write(xmin_elements_str)
        f.write(xmax_elements_str)
        f.write(ymin_elements_str)
        f.write(ymax_elements_str)
        f.write('$EndElements\n')
        

In [61]:
x_array = np.array([1, 2, 3, 4])
y_array = np.array([0, 1.5, 3])
z_array = np.array([[[0.5, 1.25, 2],
                     [0, 1, 2],
                     [0, 1, 2],
                     [0, 1, 2]],
                    [[0, 1, 2],
                     [0, 1, 2],
                     [0, 1, 2],
                     [0, 1, 2]],
                    [[0, 1, 2],
                     [0, 1, 2],
                     [0, 1, 2],
                     [0, 1, 2]]])

create_msh_from_array(x_array, y_array, z_array, 'test.msh')